In [ ]:
import pandas as pd
import os
from google.cloud import bigquery
from google.colab import auth

# Ingesta de datos

In [ ]:
# URL cruda del archivo CSV
URL = "https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/"

Confirmed_CSV = "time_series_covid19_confirmed_global.csv"
Deaths_CSV = "time_series_covid19_deaths_global.csv"
Recovered_CSV = "time_series_covid19_recovered_global.csv"

# Descargar los archivos CSV
os.system('curl -O ' + URL + Confirmed_CSV)
DF_Confirmed = pd.read_csv(Confirmed_CSV, sep=",")

os.system('curl -O ' + URL + Deaths_CSV)
DF_Deaths = pd.read_csv(Deaths_CSV, sep=",")

os.system('curl -O ' + URL + Recovered_CSV)
DF_Recovered = pd.read_csv(Recovered_CSV, sep=",")


In [ ]:
DF_Recovered.columns

Index(['Province/State', 'Country/Region', 'Lat', 'Long', '1/22/20', '1/23/20',
       '1/24/20', '1/25/20', '1/26/20', '1/27/20',
       ...
       '2/28/23', '3/1/23', '3/2/23', '3/3/23', '3/4/23', '3/5/23', '3/6/23',
       '3/7/23', '3/8/23', '3/9/23'],
      dtype='object', length=1147)

In [ ]:
DF_Deaths.head(5)

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,2/28/23,3/1/23,3/2/23,3/3/23,3/4/23,3/5/23,3/6/23,3/7/23,3/8/23,3/9/23
0,NaN,Afghanistan,33.93911,67.709953,0,0,0,0,0,0,...,7896,7896,7896,7896,7896,7896,7896,7896,7896,7896
1,NaN,Albania,41.15330,20.168300,0,0,0,0,0,0,...,3598,3598,3598,3598,3598,3598,3598,3598,3598,3598
2,NaN,Algeria,28.03390,1.659600,0,0,0,0,0,0,...,6881,6881,6881,6881,6881,6881,6881,6881,6881,6881
3,NaN,Andorra,42.50630,1.521800,0,0,0,0,0,0,...,165,165,165,165,165,165,165,165,165,165
4,NaN,Angola,-11.20270,17.873900,0,0,0,0,0,0,...,1933,1933,1933,1933,1933,1933,1933,1933,1933,1933


In [ ]:
DF_Confirmed.head(5)

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,2/28/23,3/1/23,3/2/23,3/3/23,3/4/23,3/5/23,3/6/23,3/7/23,3/8/23,3/9/23
0,NaN,Afghanistan,33.93911,67.709953,0,0,0,0,0,0,...,209322,209340,209358,209362,209369,209390,209406,209436,209451,209451
1,NaN,Albania,41.15330,20.168300,0,0,0,0,0,0,...,334391,334408,334408,334427,334427,334427,334427,334427,334443,334457
2,NaN,Algeria,28.03390,1.659600,0,0,0,0,0,0,...,271441,271448,271463,271469,271469,271477,271477,271490,271494,271496
3,NaN,Andorra,42.50630,1.521800,0,0,0,0,0,0,...,47866,47875,47875,47875,47875,47875,47875,47875,47890,47890
4,NaN,Angola,-11.20270,17.873900,0,0,0,0,0,0,...,105255,105277,105277,105277,105277,105277,105277,105277,105288,105288


In [ ]:
DF_Recovered.head(5)

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,2/28/23,3/1/23,3/2/23,3/3/23,3/4/23,3/5/23,3/6/23,3/7/23,3/8/23,3/9/23
0,NaN,Afghanistan,33.93911,67.709953,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,NaN,Albania,41.15330,20.168300,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,NaN,Algeria,28.03390,1.659600,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,NaN,Andorra,42.50630,1.521800,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,NaN,Angola,-11.20270,17.873900,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


# Transformación

In [ ]:
def melt_dataframe(df):
  #Conversión de columnas "fecha" en una columna
  df_melted = pd.melt(df,
                      id_vars=['Province/State', 'Country/Region', 'Lat', 'Long'],  # Estas columnas se mantienen
                      var_name='Date',    # Nombre de la nueva columna que contendrá las fechas
                      value_name='Count')  # Nombre de la columna que contendrá los valores de recuperados

  # Convertimos la columna 'Fecha' a formato datetime (opcional pero recomendable)
  df_melted['Date'] = pd.to_datetime(df_melted['Date'])

  return df_melted


In [ ]:
DF_Confirmed = melt_dataframe(DF_Confirmed)
DF_Deaths = melt_dataframe(DF_Deaths)
DF_Recovered = melt_dataframe(DF_Recovered)

<ipython-input-7-c1b474ff64e8>:9: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df_melted['Date'] = pd.to_datetime(df_melted['Date'])
<ipython-input-7-c1b474ff64e8>:9: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df_melted['Date'] = pd.to_datetime(df_melted['Date'])
<ipython-input-7-c1b474ff64e8>:9: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df_melted['Date'] = pd.to_datetime(df_melted['Date'])


In [ ]:
DF_Confirmed["State"] = "Confirmed"
DF_Deaths["State"] = "Deaths"
DF_Recovered["State"] = "Recovered"

DF_Covid.rename(columns={
    "Province/State": "Province_State",
    "Country/Region": "Country_Region"
}, inplace=True)

In [ ]:
DF_Covid = pd.concat([DF_Confirmed, DF_Deaths, DF_Recovered])

In [ ]:
DF_Covid.dtypes

,0
Province/State,object
Country/Region,object
Lat,float64
Long,float64
Date,datetime64[ns]
Count,int64
State,object


# Carga a BigQuery

In [ ]:
auth.authenticate_user()

project_id = 'tareamoduloviii'  # Reemplaza con tu ID de proyecto
dataset_id = 'Covid'             # Reemplaza con tu ID de dataset
table_id = 'DF_Covid'            # Nombre de la tabla donde deseas cargar los datos

# Creación cliente de BigQuery
client = bigquery.Client(project=project_id)

# Referencia de la tabla
table_ref = f"{project_id}.{dataset_id}.{table_id}"

# Cargar el DataFrame a BigQuery
try:
    job = client.load_table_from_dataframe(DF_Covid, table_ref)
    job.result()  # Esperar a que la carga se complete
    print(f"Cargados {job.output_rows} filas en la tabla {table_ref}.")

except Exception as e:

    print(f"Error al cargar los datos: {e}")


Cargados 973836 filas en la tabla tareamoduloviii.Covid.DF_Covid.


In [ ]:
DF_Covid.to_csv('DF_Covid.csv', index=False)